# 270 : M2 Demo and qualitative analysis

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

0


In [2]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR').resolve()
  OUT_BASE = Path('/work/stual/res_ICDAR/method_2').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_2


### Constants

In [3]:
### Model to run
MODEL_PATH = OUT_BASE / 'tmp'

FINETUNED_MODEL = MODEL_PATH / "221-camembert-ner-joint-labelling-io/checkpoint-1800"
FINETUNED_MODEL

PosixPath('/work/stual/res_ICDAR/method_2/tmp/221-camembert-ner-joint-labelling-io/checkpoint-1800')

In [4]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

LIMIT = 100

tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL) #Param : tokenizer du modele souhaité
model = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL) #Modèle choisi

#Classification des entités
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None, use_fast=True)

2023-02-22 09:15:43.928319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 09:15:45.287235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 09:15:45.287334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-22 09:15:45.287344: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/work/stual/res_ICDAR/method_2/tmp/221-camembert-ner-joint-labelling-io/checkpoint-1800'. Use `repo_type` argument if needed.

### Examples

In [7]:
samples = """
☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO \n
Dutay, essayeur du commerce, place Dau-  - píině, 5. DOB \n
Dulay, chandronnier, r. du Pont- aux- Chuux, 15. SI \n
Dufay (V.e), grenetière, r. du Fauh.-S. Denis, 20. 372 \n
Y ☞ Dnten,charentier, 1. Montmartre, 89. 318 \n
Dufey fils, bijoutier, passage de la Réunion A \n
Dnley, boucher, r. S.&quot;Louis-au-Marais,3. * \n
☞ Duffand, bijonteer, r. S.-Martin, 107. 284 \n
Duftaud, maçon, r. S-Honore  353. O \n
Dullaut, chandronnier, r. de la Sourdière I- ( ☞T4 \n
lukils. limonauier, 1. des Colonnes, 6. J9 \n
— T Dullocu, architecte, r. Basse-Porte-S.De- — Ris, 28. \n
Duflot, loueur de carrosses, r. de Paradis- 505 Poissonnière, 22. \n
Dufort, bottier, Palais-R., gal. vitrée, 215. A \n
Dufort fils, bottier (exp. 1819.), r. J.-J.. Rousscan. 18. 290 \n
* Dufort, tabletier, r. Jean-Pain-Mollet, 10. 4O \n
Dufossé, teiniurier, r. du Faub.-Montmar- tre , 63. 483 \n
Duſour el Besnard, march, de bois à bruler, quai de la Tournelle, 17. etr. des Fossés- SBernard. 11. Dí \n
Dufour jeune, march, de bois à brüler de la Pépinière, 53. 7 r. \n
Dufour (Mad.), march. de chapeaux de paitle, passage dn Panorama, 11. IO \n
Dutour, thapelier, r. S.-Honore 383. 3og \n
Dufour, cirier, r. de PArbre-Sec, 6o. 316 \n
Dufour et co., commiss., r. des Contures-  DAO S. Dervajs, 18. \n
Dulour, corropeir, 1. des Vertusy1d 3 \n
Dufour (Charles), épicier, r. S.-Denis, Jo7. ☞ 332 \n
Dufour, fab. d&apos;éventails (erp. 1819) Beaubourg, 48.  360 \n
Dufour, faiencter, r. S.-Anne , 50. 302 \n
Dufour, A. Sarnt,r. Ventadour, ☞7 \n
Dufour (Gabriel), libraire, r. de Vaugirard, A \n
Dufour, maçon, r. S.-Jean-Baptiste, 4 402 \n
Dufour, architecte, r. de Chartres-S.-Ho- 2781 40a nore, 12. (Elig.) \n
Dufour (Mad.), nouveautes, 1. Neuve-des- H12 Petits-Champs, 4. \n
Dufour Chabrol, papetier, r. S.-Martin ☞ 4 \n
Dufour (J.) et co,, fab. de papiers peints, ((1819); r. Beauvean-S. Antoine, 19. \n
Waring et Gillow Limited, ameublements an- glais, éctairage électrique et appareils sa- nitaires, r. Glück, 8 et boul. Haussmann, 31, TELÉPH] 244.45; succursale, r. Bleue, 7. WARISON�PARIS. TĚLÉPH 244.40. Adr.\n
Barruel (Paul), fabr. de produis chimiques et de siccatif zumatique, adjoint au maire du 15e arrond., 
Alleray, 70; dépót chez H. Suillot sle-Croix-de-la-Bretonnerič. 21 \n
Marchand (Mme) née Louise Scelle, en gros et détail, r. 77, et à Bercy, r. de Bourgogne, 25. d'Enfer, \n
Very (A.). ingenieur-constructeur, r. d'Angou- lême, 66, cité d'Angouléme, 6. TĚLÉPH 420.82. \n
Vacherie de Royaumont, Brémontler. 16; Chaussée-d'Antin, 68; Phals- bourg, 3 ; blv. Poissonnière, 1, et blv. St-Martin, 15. \n
Budan, inspect.-gén. des études, r. Charlot. 18 (au Marais). \n
"""

In [9]:
from xmlize_util import get_NER_tags, xmlize_multilevel

for s in samples.split('\n'):
    if s != '':
        print(s)
        res = nlp(s)
        print(res)
        levels, num_levels = get_NER_tags(s,res)
        xml = xmlize_multilevel(levels,num_levels)
        #Corrections IO
        """
        - Sur-apprentissage : . = fin d'entité ? => </LOC></SPAT>.<SPAT><LOC>
        - Classification de déchets de l'OCR : 
            - ex : <LOC> JO</LOC>
            - Différence floue entre déchet de l'OCR et CARDINAL
            - <SPAT><LOC> r. de Chartres-S.-Ho- 27</LOC><CARDINAL>81 40</CARDINAL><LOC>a n</LOC></SPAT>
        - Mauvais découpage dans une entité : <PER><TITRE-H>☞</TITRE-H> Duff</PER>and     <PER> Du</PER>fort
        - Ponctuation : <PER>),</PER>
        - ??? : <ACT> commis</ACT><PER>s</PER>
        - Confusion TITRE, TITRE-H (assez logique)
        - Passe à côté des TITRE avec le PSEP : <PER>Y<TITREH> </TITREH></PER>☞<PER> Dnten</PER> (entraine un décallage dans la fonction de reconstruction XML)
        - Ressort 2 fois le même morceau de texte dans deux classes différentes : <PER><TITREH>☞</TITREH></PER>☞<PER> Duffand</PER> ???? Problèmes fonction xml ?
        - Coupe des mots en plusieurs parties : <SPAT><LOC> r. de Paradis-</LOC><CARDINAL> 505</CARDINAL><LOC> Poisson</LOC>nière
        - Erreur d'encodage post-traitement : </ACT>&amp;<ACT> pour le &
        
        xml = xml.replace('</LOC></SPAT>.<SPAT><LOC>','.')
        xml = xml.replace('</LOC></SPAT><DESC>.</DESC><SPAT><LOC>','.')
        xml = xml.replace('</ACT>&amp;<ACT>','&')
        xml = xml.replace('</ACT>.<ACT>','.')
        xml = xml.replace('</LOC><CARDINAL>.</CARDINAL><LOC>','.')
        xml = xml.replace("</LOC></SPAT><ACT>'</ACT><SPAT><LOC>","'")
        xml = xml.replace("<LOC>,</LOC>",',')
        xml = xml.replace("<LOC>.</LOC>",'.')
        xml = xml.replace("<CARDINAL>.</CARDINAL>",".")
        xml = xml.replace("<SPAT>.</SPAT>",".")
        """
        print(xml)
        print('')

☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO 
[{'entity': 'O+O', 'score': 0.99571, 'index': 1, 'word': '▁', 'start': 0, 'end': 1}, {'entity': 'O+O', 'score': 0.99648464, 'index': 2, 'word': '☞', 'start': 0, 'end': 1}, {'entity': 'O+O', 'score': 0.9963676, 'index': 3, 'word': '▁T', 'start': 2, 'end': 4}, {'entity': 'I-PER+O', 'score': 0.99819547, 'index': 4, 'word': '▁Du', 'start': 4, 'end': 7}, {'entity': 'I-PER+O', 'score': 0.9981883, 'index': 5, 'word': 'fant', 'start': 7, 'end': 11}, {'entity': 'I-PER+O', 'score': 0.998181, 'index': 6, 'word': '▁(', 'start': 11, 'end': 13}, {'entity': 'I-PER+O', 'score': 0.9981736, 'index': 7, 'word': 'Vi', 'start': 13, 'end': 15}, {'entity': 'I-PER+O', 'score': 0.9981623, 'index': 8, 'word': 'ctor', 'start': 15, 'end': 19}, {'entity': 'O+O', 'score': 0.9929508, 'index': 9, 'word': '),', 'start': 19, 'end': 21}, {'entity': 'I-ACT+O', 'score': 0.9947491, 'index': 10, 'word': '▁libraire', 'start': 21, 'end': 30}, {'entity': 'O+O', 'score

[{'entity': 'O+O', 'score': 0.99695694, 'index': 1, 'word': '▁', 'start': 0, 'end': 1}, {'entity': 'O+O', 'score': 0.9970897, 'index': 2, 'word': '☞', 'start': 0, 'end': 1}, {'entity': 'I-PER+O', 'score': 0.9981906, 'index': 3, 'word': '▁D', 'start': 1, 'end': 3}, {'entity': 'I-PER+O', 'score': 0.99818134, 'index': 4, 'word': 'uff', 'start': 3, 'end': 6}, {'entity': 'I-PER+O', 'score': 0.99817324, 'index': 5, 'word': 'and', 'start': 6, 'end': 9}, {'entity': 'O+O', 'score': 0.9872393, 'index': 6, 'word': ',', 'start': 9, 'end': 10}, {'entity': 'I-ACT+O', 'score': 0.9961438, 'index': 7, 'word': '▁bi', 'start': 10, 'end': 13}, {'entity': 'I-ACT+O', 'score': 0.99613404, 'index': 8, 'word': 'jon', 'start': 13, 'end': 16}, {'entity': 'I-ACT+O', 'score': 0.9960602, 'index': 9, 'word': 'te', 'start': 16, 'end': 18}, {'entity': 'I-ACT+O', 'score': 0.99608755, 'index': 10, 'word': 'er', 'start': 18, 'end': 20}, {'entity': 'O+O', 'score': 0.9959798, 'index': 11, 'word': ',', 'start': 20, 'end': 2

[{'entity': 'I-PER+O', 'score': 0.99819297, 'index': 1, 'word': '▁Du', 'start': 0, 'end': 2}, {'entity': 'I-PER+O', 'score': 0.99818987, 'index': 2, 'word': 'fort', 'start': 2, 'end': 6}, {'entity': 'I-PER+O', 'score': 0.99818987, 'index': 3, 'word': '▁fils', 'start': 6, 'end': 11}, {'entity': 'O+O', 'score': 0.97874534, 'index': 4, 'word': ',', 'start': 11, 'end': 12}, {'entity': 'I-DESC+i_ACT', 'score': 0.9174126, 'index': 5, 'word': '▁bot', 'start': 12, 'end': 16}, {'entity': 'I-DESC+i_ACT', 'score': 0.92170715, 'index': 6, 'word': 'tier', 'start': 16, 'end': 20}, {'entity': 'I-DESC+i_ACT', 'score': 0.85495996, 'index': 7, 'word': '▁(', 'start': 20, 'end': 22}, {'entity': 'I-DESC+i_TITREP', 'score': 0.8442012, 'index': 8, 'word': 'exp', 'start': 22, 'end': 25}, {'entity': 'I-DESC+i_TITREP', 'score': 0.7632165, 'index': 9, 'word': '.', 'start': 25, 'end': 26}, {'entity': 'I-DESC+i_TITREP', 'score': 0.85508275, 'index': 10, 'word': '▁18', 'start': 26, 'end': 29}, {'entity': 'I-DESC+i_

[{'entity': 'I-PER+O', 'score': 0.99819154, 'index': 1, 'word': '▁Du', 'start': 0, 'end': 2}, {'entity': 'I-PER+O', 'score': 0.99818414, 'index': 2, 'word': 'four', 'start': 2, 'end': 6}, {'entity': 'I-PER+O', 'score': 0.9981812, 'index': 3, 'word': '▁jeune', 'start': 6, 'end': 12}, {'entity': 'O+O', 'score': 0.9871601, 'index': 4, 'word': ',', 'start': 12, 'end': 13}, {'entity': 'I-DESC+i_ACT', 'score': 0.66297853, 'index': 5, 'word': '▁march', 'start': 13, 'end': 19}, {'entity': 'I-DESC+i_ACT', 'score': 0.59895146, 'index': 6, 'word': ',', 'start': 19, 'end': 20}, {'entity': 'I-DESC+i_ACT', 'score': 0.65066147, 'index': 7, 'word': '▁de', 'start': 20, 'end': 23}, {'entity': 'I-DESC+i_ACT', 'score': 0.5357219, 'index': 8, 'word': '▁bois', 'start': 23, 'end': 28}, {'entity': 'I-DESC+i_ACT', 'score': 0.5806365, 'index': 9, 'word': '▁à', 'start': 28, 'end': 30}, {'entity': 'I-DESC+i_ACT', 'score': 0.52131927, 'index': 10, 'word': '▁br', 'start': 30, 'end': 33}, {'entity': 'I-DESC+i_ACT', 

<PER>Dufour</PER>,<DESC><ACT> fab. d</ACT></DESC>&amp;<DESC><ACT>apos;éventails</ACT> (<TITREP>erp. 1819</TITREP>)</DESC><SPAT><LOC> Beaubourg</LOC>,<CARDINAL> 48</CARDINAL></SPAT>.  360 

Dufour, faiencter, r. S.-Anne , 50. 302 
[{'entity': 'I-PER+O', 'score': 0.9981838, 'index': 1, 'word': '▁Du', 'start': 0, 'end': 2}, {'entity': 'I-PER+O', 'score': 0.99817884, 'index': 2, 'word': 'four', 'start': 2, 'end': 6}, {'entity': 'O+O', 'score': 0.9402022, 'index': 3, 'word': ',', 'start': 6, 'end': 7}, {'entity': 'I-ACT+O', 'score': 0.9962482, 'index': 4, 'word': '▁fai', 'start': 7, 'end': 11}, {'entity': 'I-ACT+O', 'score': 0.9961845, 'index': 5, 'word': 'enc', 'start': 11, 'end': 14}, {'entity': 'I-ACT+O', 'score': 0.99601114, 'index': 6, 'word': 'ter', 'start': 14, 'end': 17}, {'entity': 'O+O', 'score': 0.9914961, 'index': 7, 'word': ',', 'start': 17, 'end': 18}, {'entity': 'I-SPAT+i_LOC', 'score': 0.9982121, 'index': 8, 'word': '▁r', 'start': 18, 'end': 20}, {'entity': 'I-SPAT+i_LOC', '

[{'entity': 'I-PER+O', 'score': 0.9981901, 'index': 1, 'word': '▁Du', 'start': 0, 'end': 2}, {'entity': 'I-PER+O', 'score': 0.99818856, 'index': 2, 'word': 'four', 'start': 2, 'end': 6}, {'entity': 'I-PER+O', 'score': 0.99816495, 'index': 3, 'word': '▁(', 'start': 6, 'end': 8}, {'entity': 'I-PER+O', 'score': 0.9981573, 'index': 4, 'word': 'J', 'start': 8, 'end': 9}, {'entity': 'I-PER+O', 'score': 0.9979956, 'index': 5, 'word': '.', 'start': 9, 'end': 10}, {'entity': 'I-PER+O', 'score': 0.9980573, 'index': 6, 'word': ')', 'start': 10, 'end': 11}, {'entity': 'I-PER+O', 'score': 0.9980579, 'index': 7, 'word': '▁et', 'start': 11, 'end': 14}, {'entity': 'I-PER+O', 'score': 0.99714065, 'index': 8, 'word': '▁co', 'start': 14, 'end': 17}, {'entity': 'O+O', 'score': 0.9959488, 'index': 9, 'word': ',', 'start': 17, 'end': 18}, {'entity': 'O+O', 'score': 0.9562572, 'index': 10, 'word': ',', 'start': 18, 'end': 19}, {'entity': 'I-DESC+i_ACT', 'score': 0.9461617, 'index': 11, 'word': '▁fa', 'start'

[{'entity': 'I-PER+O', 'score': 0.99818903, 'index': 1, 'word': '▁Marchand', 'start': 0, 'end': 8}, {'entity': 'I-PER+O', 'score': 0.99815875, 'index': 2, 'word': '▁(', 'start': 8, 'end': 10}, {'entity': 'I-PER+O', 'score': 0.99814117, 'index': 3, 'word': 'M', 'start': 10, 'end': 11}, {'entity': 'I-PER+O', 'score': 0.9981511, 'index': 4, 'word': 'me', 'start': 11, 'end': 13}, {'entity': 'I-PER+O', 'score': 0.9981285, 'index': 5, 'word': ')', 'start': 13, 'end': 14}, {'entity': 'I-PER+O', 'score': 0.99802387, 'index': 6, 'word': '▁née', 'start': 14, 'end': 18}, {'entity': 'I-PER+O', 'score': 0.9980819, 'index': 7, 'word': '▁Louise', 'start': 18, 'end': 25}, {'entity': 'I-PER+O', 'score': 0.998038, 'index': 8, 'word': '▁Sc', 'start': 25, 'end': 28}, {'entity': 'I-PER+O', 'score': 0.9980046, 'index': 9, 'word': 'elle', 'start': 28, 'end': 32}, {'entity': 'O+O', 'score': 0.9958585, 'index': 10, 'word': ',', 'start': 32, 'end': 33}, {'entity': 'I-ACT+O', 'score': 0.99148434, 'index': 11, 'w